In [4]:
import pandas as pd
import conllu as cl
from os.path import join

In [17]:
def load(language):
    filePath = ".\Datasets\\" + language + "\\"
    
    with open(join(filePath, "train.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    train_sentences = cl.parse(data)

    with open(join(filePath, "test.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    test_sentences = cl.parse(data)

    with open(join(filePath, "val.conllu"), 'r', encoding='utf-8') as file:
        data = file.read()
    val_sentences = cl.parse(data)

    return train_sentences, test_sentences, val_sentences

In [18]:
it_train_sentences, it_test_sentences, it_val_sentences = load("it")
#en_train_sentences, en_test_sentences, en_val_sentences = load("en")
#es_train_sentences, es_test_sentences, es_val_sentences = load("es")

[TokenList<Si, stabilì, ad, Amburgo, per, la, sua, ammirazione, nei, confronti, della, letteratura, tedesca, (, aveva, imparato, la, lingua, in, carcere, ), ,, specialmente, per, i, romantici, come, Novalis, e, Hölderlin, .>, TokenList<Anche, i, veicoli, interplanetari, utilizzano, principalmente, i, razzi, chimici, ,, anche, se, alcuni, hanno, usato, sperimentalmente, con, successo, i, nuovi, propulsori, ionici, (, come, ad, esempio, il, satellite, dell', ESA, SMART-1, ), .>, TokenList<Dal, 1866, al, 1869, egli, frequentò, il, liceo, di, recente, costruzione, a, Arnhem, ,, e, nel, 1870, egli, passò, gli, esami, sulle, lingue, classiche, necessario, per, l', ammissione, all', università, .>, TokenList<Una, strada, militare, venne, costruita, attraverso, le, colline, che, vanno, da, Vladikavkaz, fino, a, Tbilisi, .>, TokenList<Raggiunge, la, popolarità, nel, 1988, nel, varietà, televisivo, ", Cocco, ", di, Rai, 2, con, il, personaggio, dell', automobilista, ", incazzato, come, una, best